# Вероятность

## Условная вероятность

Пусть есть два события $A$ и $B$. Тогда условная вероятность вводится следующим образом(аксиоматически):

$$p(A|B) = \frac{p(A, B)}{p(B)}$$

Которое показывает какая вероятность наступить событию $A$, если событие $B$ уже наступило.

Если $p(A|B)=p(A)$, то говорят что событие $A$ не зависит от $B$.

## Формула Байеса

Формула Байеса выводится из формулы условной вероятности в два щелчка:

$$p(A|B) = \frac{p(A, B)}{p(B)} = \frac{p(A) p(B|A)}{p(B)}$$

Мы к ней ещё вернёмся.

# Задача 0.

В семье два ребёнка. Каждый ребёнок может быть или мальчиком, или девочкой с равной вероятностью 50%.

Тогда полное пространство событий описывается четырьмя событиями.

![](full_space_boy_girl.png)

#### Если известно что в семье два ребёнка, то какая вероятность, что оба мальчики?

$$p(bb) = ?$$

![](two_boys.png)

#### Если известно что в семье два ребёнка и один из них мальчик, то какая вероятность, что другой тоже мальчик?

$$p(bb | b) = ?$$

$p(b)$ - вероятность что хотя бы один мальчик.

![](one_is_a_boy.png)

#### Если известно что в семье два ребёнка и старший из них мальчик, то какая вероятность, что младший тоже мальчик?

$$p(bb | b_{old} ) = ?$$

Разница между предыдущим в том что мы определили порядок на парах (мальчик/девочка, мальчик/девочка), что сильно влияет на вероятности(см. картинку).

![](old_is_boy.png)

#### Если известно что в семье два ребёнка и один из них мальчик родившийся во вторник, то какая вероятность, что второй тоже мальчик?

$$p(bb | b_{tuesday} ) = ?$$

![](tuesday_boy.png)

#### Мы проходили мимо дома семьи и увидели в окне мальчика, какая вероятность что второй ребёнок тоже мальчик?

$$p(b) = \frac{1}{2}$$

![](one_is_a_boy_obs.png)

Важный момент! 

Априорное знание что один ребёнок мальчик не тоже самое что пронаблюдать что один из детей мальчик.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=2)

# 2. Жизненная задачка

По Дарвину приспособленность является центральной концепцией теории эволюции. 

Давайте определим "относительную приспособленность" как среднее количество выживших представителей одного генотипа к среднему количеству выживших представителей другого генотипа. 

Eyre-Walker (2006) ( https://www.ncbi.nlm.nih.gov/pubmed/16547091 ) предположили, что функция преспособленности принадлежит семейству гамма-распределений случайных величин. 

Они проверили своё предположение для популяций(кого?) с мутацией `deleterious amino acid`(что это значит?). 

Они получили следующие оценки на параметры гамма-распределения:
 
$$\hat{\alpha} = 0.23,~~~ \hat{\beta} = 5.35$$


$$\Gamma(x|\alpha, \beta) =  \frac{\beta^\alpha}{\Gamma(\alpha)} x^{\alpha-1} \exp(-\beta x),$$

где $\Gamma(\alpha)$ -- гамма-функция Эйлера(для целых $\alpha$: $\Gamma(k) = (\alpha-1)!$)

#### Зададимся следующим вопросом

Если мы оцениваем параметры $\alpha$ и $\beta$ по случайной выборке, как сильно мы в среднем ошибаемся при размере выборки $N=100$? 

Т.е. нас интересуют две вещи: __дисперсии__ наших оценок и __корреляция__ оценок.

Оценивать мы будем их по следующему алгоритму:

  1. Сэмплируем выборку $X \sim \Gamma(x|\hat{\alpha}, \hat{\beta})$
  2. Оцениваем $\alpha_{fit}, \beta_{fit}$ по выборке $X$
  3. Сохраняем $\alpha_{fit}, \beta_{fit}$
  4. Повторяем шаги 1-3 несколько тысяч раз
  5. Считаем дисперсии и корреляции $\{\alpha_{fit}, \beta_{fit}\}_{i=1}^{B}$

## 2.1 Метод оценки параметров максимума правдоподобия

MLE(Метод оценки параметров максимума правдоподобия) основывается на максимизации вероятности пронаблюдать выборку.

$$\mathcal{L} = \prod p(x_i|\theta)$$

Будем рассматривать MLE на примере задачи выше.

In [ ]:
from scipy.stats import gamma

In [ ]:
# scale == \theta
# в scipy все распределения определены с точность до смещения и масштаба
x = np.linspace(0, 1, 100)
alpha = 0.23
beta = 5.35
plt.plot(gamma.pdf(x, scale=1 / beta, a=alpha))

In [ ]:
np.random.seed(1337)
X = gamma.rvs(scale=1 / beta, a=alpha, size=100)

In [ ]:
gamma.pdf(X[0], scale=1 / beta, a=alpha)

In [ ]:
np.prod(gamma.pdf(X, scale=1 / beta, a=2.5))

Уравнение правдоподобия чаще всего записывается с использованием логарифма вероятности, так как в этом случае произведение переходит в сумму, что сильно упрощает жизнь.

$$\mathcal{\log L} = \sum\limits_i \log p(x_i|\theta)$$

  * более хорошая численная стабильность(предотвращает overflow или underflow ошибки);
  * более (относительно)точные градиенты;
  * более гладкая задача(методы оптимизации лучше работают).

Посмотрим на зависимость log-likelihood для нашей задачи с гамма-распределением.

$$\mathcal{L} = \prod\limits_i \frac{\beta^\alpha}{\Gamma(\alpha)} {x_i}^{\alpha-1} \exp(-\beta x_i)$$

$$\mathcal{\log L} = \sum\limits_i \left(  \alpha \log \beta - \log \Gamma(\alpha) - \beta x_i + (\alpha - 1) \log x_i \right)$$

(Обратим внимание насколько простой стала зависимость от $x_i$:)

In [ ]:
a = np.linspace(0, 1, 200)
plt.plot(
    a, 
    np.prod(gamma.pdf(X[:, np.newaxis], 
                      scale=1 / beta, 
                      a=a), 
            axis=0)
)
plt.title('Likelihood')

In [ ]:
plt.plot(
    a, 
    np.sum(gamma.logpdf(X[:, np.newaxis], 
                        scale=1 / beta, 
                        a=a), 
            axis=0)
)
plt.title('Log-Likelihood')

### MLE? MLE!

В большинстве задач, если это не игровые примеры, найти явную форму для оценки параметров с помощью MLE очень сложно.

$$\theta^* = \arg \max \log \mathcal{L}(X, \theta)$$

<Вывести MLE для гамма-распределения>

Увы, точного решения нет. 

Но есть кое-что лучше(и более применимое на практике)

In [ ]:
gamma.fit?

In [ ]:
alpha, _, beta = gamma.fit(X, floc=0.)
beta = 1 / beta
print(alpha, beta)

## Пришло время эксперимента :)

In [ ]:
%%time
boot = 100000
alphas = []
betas = []
fitted_alphas = []
fitted_betas = []
for i in range(boot):
    <YOUR CODE>
    fitted_betas.append(1 / fit_beta)
    fitted_alphas.append(fit_alpha)

In [ ]:
plt.hist(fitted_alphas, bins=100);
alpha_q5, alhpa_q95 = np.percentile(fitted_alphas, [5, 95])
plt.axvline(x=alpha, linewidth=3, color='r',linestyle='--')
plt.axvline(x=alpha_q5, linewidth=3, color='g',linestyle='--')
plt.axvline(x=alhpa_q95, linewidth=3, color='g',linestyle='--')

In [ ]:
np.mean(np.array(fitted_alphas) - alpha)

In [ ]:
plt.hist(fitted_betas, bins=100);
beta_q5, beta_q95 = np.percentile(fitted_betas, [5, 95])
plt.axvline(x=beta, linewidth=3, color='r',linestyle='--')
plt.axvline(x=beta_q5, linewidth=3, color='g',linestyle='--')
plt.axvline(x=beta_q95, linewidth=3, color='g',linestyle='--')

In [ ]:
np.mean(np.array(fitted_betas) - beta)

In [ ]:
plt.scatter(fitted_alphas, fitted_betas)

In [ ]:
np.corrcoef(fitted_alphas, fitted_betas)

In [ ]:
from scipy.stats.stats import pearsonr
pearsonr(fitted_alphas, fitted_betas)

## 2.2 Метод моментов

Идея простая. Если мы хотим определить параметры некоторого распределения $p(x | \theta)$, то давайте приравняем моменты данного распределения к моментам подсчитанным по данным($X = \{ x_1, \dots, x_n \}$) и решим уравнения относительно данных моментов.

Моменты по данным считаются следующим образом:

$$\hat{\mu}_k = \frac{1}{n} \sum\limits_{i=1}^{n} x_i^k$$

Моменты распределений:

$$\mu_k = \sum\limits_{i=1}^{n} x^k p(x | \theta) dx$$

In [ ]:
from scipy.stats import gamma
# scale == \theta
# в scipy все распределения определены с точность до смещения и масштаба
x = np.linspace(0, 1, 100)
alpha = 0.23
beta = 5.35
plt.plot(gamma.pdf(x, scale=1 / beta, a=alpha))

Среднее и дисперсия гамма-распределения:

$$\mu(\Gamma) = \frac{\alpha}{\beta}$$

$$Var(\Gamma) = \frac{\alpha}{\beta^2}$$

Применяем метод моментов:

<Вывести оценки на параметры с помощью МоМ>

In [ ]:
%%time
boot = 100000
fitted_alphas = []
fitted_betas = []
for i in range(boot):
    X = gamma.rvs(scale=1 / beta, a=alpha, size=100)
    mu_1 = <YOUR CODE>
    mu_2 = <YOUR CODE>
    fitted_betas.append(
        <YOUR CODE>
    )
    
    fitted_alphas.append(
        <YOUR CODE>
    )

In [ ]:
plt.hist(fitted_alphas, bins=100);
alpha_q5, alhpa_q95 = np.percentile(fitted_alphas, [5, 95])
plt.axvline(x=alpha, linewidth=3, color='r',linestyle='--')
plt.axvline(x=alpha_q5, linewidth=3, color='g',linestyle='--')
plt.axvline(x=alhpa_q95, linewidth=3, color='g',linestyle='--')

In [ ]:
np.mean(np.array(fitted_alphas) - alpha)

In [ ]:
plt.hist(fitted_betas, bins=100);
beta_q5, beta_q95 = np.percentile(fitted_betas, [5, 95])
plt.axvline(x=beta, linewidth=3, color='r',linestyle='--')
plt.axvline(x=beta_q5, linewidth=3, color='g',linestyle='--')
plt.axvline(x=beta_q95, linewidth=3, color='g',linestyle='--')

In [ ]:
np.mean(np.array(fitted_betas) - beta)

In [ ]:
plt.scatter(fitted_alphas, fitted_betas)

In [ ]:
from scipy.stats.stats import pearsonr
np.corrcoef(fitted_alphas, fitted_betas)

In [ ]:
pearsonr(fitted_alphas, fitted_betas)

## Выводы

Что мы можем сказать, сравнив метод моментов и MLE оценки? Как они соотносятся?

# 3. MAP

Предположим, что вы подбросили моменту один раз и у вас выпал орёл. 

Распределение описывается распредлением Бернулли: $p(x=1, q) = q$.

Для выборки $x_1, x_2, \dots, x_N$ правдоподобие записывается следующим образом:

$$p(X, q) = ?$$

$$\log p(X, q) = ?$$

Тогда оценка на параметры высчитывается следующим образом:

$$\frac{\partial }{\partial q} \log p(X, q) = ?$$

Получаем:

$$q = ?$$

В согласии с методом максимума правдоподобия следует, что $p=1$, т.е. следует что монетка всегда будет выпадать орлом.

Такая оценка не очень хорошо согласуется с реальностью. 

Однако у вас есть некоторое априорное знание. Вы точно знаете, что честная монетка выпадает орлом в 50% случаев.

Можем ли мы как-то это использовать? Да, это обеспечивается введением априорного распределения.

В MLE оценка параметров выглядела так:

$$\theta = \mathrm{argmax} \log P(X | \theta) = \mathrm{argmax} \sum \log p(x_i, \theta) $$


В MAP мы говорим, что есть ещё распределение $P(\theta)$, которое появляется из некоторых наших знаний о мире. Тогда MAP записывается следующим образом:

$$
\theta = \mathrm{argmax}  P(X | \theta) P(\theta) = \mathrm{argmax} \left( \sum  \log p(x_i, \theta) \right) P(\theta)
$$

Вернёмся к монете.

Априорное распределени моделируется бета-распределением у которого плотность вероятности выглядит так:

![betapdf](https://wikimedia.org/api/rest_v1/media/math/render/svg/125fdaa41844a8703d1a8610ac00fbf3edacc8e7)

![beta](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Beta_distribution_pdf.svg/531px-Beta_distribution_pdf.svg.png)

В таком случае:

$$p(X, q) p(q) = ?$$

Лог-вероятность:

$$\log p(X, q) p(q) = ? $$

Производная:


$$\frac{\partial }{\partial q} \log p(X, q) p(q) = ?$$

Решая уравнение выше получаем:

$$\mu = ?$$


Пусть наш приор $\alpha=\beta=2$(см. картинку выше).

Тогда при одном броске монеты получаем следующую оценку:

$$q = ?$$

# Бонусная задача

Выше мы обсудили решили с фитированием функции приспособленности, которая выражается гамма-распределением.

Теперь мы посмотрим на задачу с другой 